<a href="https://colab.research.google.com/github/anitakumar/ml/blob/main/measuredofAcuuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357280 sha256=a92345bf13864f8d38066da2b9c6c7afa1ec2f87fbd1fe85ea6617457a13d88f
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [17]:
# prompt: import pandas as pd

import pandas as pd
from surprise import SVD
from surprise import KNNBaseline
from surprise.model_selection import train_test_split
from surprise.model_selection import LeaveOneOut # typo fix: Leavout -> LeaveOneOut
from surprise import Reader
from surprise import Dataset
from surprise import accuracy
from collections import defaultdict


movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [15]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [18]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [19]:
df = pd.merge(movies, ratings, on='movieId',how='inner') # Changed 'movie_id' to 'movieId'

In [20]:
reader=Reader(rating_scale=(0.5,5))
data=Dataset.load_from_df(df[['userId','movieId','rating']],reader)
trainset,testset=train_test_split(data,test_size=0.25, random_state=0)

In [21]:
algo = SVD(random_state=0)

In [22]:
algo.fit(trainset)

In [23]:
prediction=algo.test(testset)

In [24]:
def MAE(prediction):
    return accuracy.mae(prediction,verbose=True)

In [25]:
def RMSE(prediction):
    return accuracy.rmse(prediction,verbose=True)

In [26]:
print("MAE: ", MAE(prediction))
print("RMSE: ", RMSE(prediction))

MAE:  0.6696
MAE:  0.6695566522267671
RMSE: 0.8722
RMSE:  0.8721506370755425


In [28]:
def getTopTen(predictions,n=10, minimum_rating=4.0):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        if (est >= minimum_rating):
            top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

In [29]:
LOOCV =LeaveOneOut(n_splits=1, random_state=0)
for trainset, testset in LOOCV.split(data):
    algo.fit(trainset)
    leftout_predictions = algo.test(testset)
    all_predictions = algo.test(trainset.build_testset())
    top_ten_predictions = getTopTen(all_predictions, n=10)
    print(top_ten_predictions)

defaultdict(<class 'list'>, {1: [(527, 5), (608, 5), (923, 5), (1197, 5), (1208, 5), (2329, 5), (50, 4.999205664637019), (47, 4.998247387622192), (2529, 4.984007685932664), (1073, 4.980124601808054)], 5: [(527, 4.396819931971824), (247, 4.3196780517574975), (50, 4.287167111931873), (475, 4.241161048481514), (318, 4.237095781544985), (296, 4.234574244706994), (58, 4.076980325298417), (110, 4.065346374707569), (34, 4.060475785496112)], 7: [(750, 4.492228617296725), (4993, 4.336900385674936), (2019, 4.2904884287640686), (5952, 4.268287998282956), (5618, 4.25454056945366), (1219, 4.227338562307955), (2762, 4.174993184921616), (260, 4.162735666857165), (1240, 4.161986608296267), (1196, 4.121605449215488)], 15: [(260, 4.47865379602586), (318, 4.34861493273398), (1196, 4.327104867581584), (1200, 4.240804983319598), (3147, 4.2366709564207), (589, 4.235945375884567), (527, 4.189330880724475), (1270, 4.184131994716108), (1210, 4.180847556485074), (356, 4.169173740757654)], 17: [(318, 4.659615255

In [33]:
def HitRate(top_ten_predictions, leftout_predictions):
    hits = 0
    total = 0
    # Iterate through each left-out prediction
    for leftout in leftout_predictions:
        user_id = leftout[0]
        leftout_movie_id = leftout[1]
        hit = False
        # Check if the user has any predictions in the top_ten_predictions
        if user_id in top_ten_predictions:
            # Iterate through the top ten movie predictions for the user
            for movie_id, predicted_rating in top_ten_predictions[user_id]:
                if leftout_movie_id == movie_id:
                    hit = True
                    break
        if hit:
            hits += 1
        total += 1
    # Calculate and return the hit rate
    return hits/total

In [34]:
print("Hit Rate: ", HitRate(top_ten_predictions, leftout_predictions))

Hit Rate:  0.0
